<a href="https://colab.research.google.com/github/miguelmota2301/ciencia_de_dados/blob/main/Pr%C3%A1tica_5_Subset_Selection_e_Regulariza%C3%A7%C3%A3o_23_05_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autor: Miguel Feliciano Mota Alves

# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

A função `combinations` será responsável por fazer as combinações necessárias para o subset selection.

Documentação: [itertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations)

In [ ]:
from itertools import combinations

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

Veremos adiante como utilizar a validação cruzada com dados padronizados.

Documentação: [Pipeline()](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import Pipeline

# Data Set

Usaremos o banco de dados [Abalone](https://www.kaggle.com/datasets/nayanack/abalone-dataset-from-uci/data?select=abalone1.data.txt), disponível no [UCI Repository](https://archive.ics.uci.edu/dataset/1/abalone). Nesse banco de dados, temos algumas informações do animal abalone (sexo, largura, diâmetro, altura, ...) a fim de descubrir a idade do animal (rings).

Recomendo baixar do link [Abalone](https://www.kaggle.com/datasets/nayanack/abalone-dataset-from-uci/data?select=abalone1.data.txt) e fazer o upload no drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/archive/abalone1.data.txt')
df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight\t,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [ ]:
df.isnull().sum()

Sex                0
 Length            0
 Diameter          0
 Height            0
 Whole_weight      0
 Shucked_weight    0
 Viscera_weight    0
 Shell_weight\t    0
 Rings             0
dtype: int64

In [ ]:
df = pd.get_dummies(df, columns = ['Sex'])
df

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight\t,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,False,False,True
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,False,False,True
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,True,False,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,False,False,True
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,True,False,False
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,False,False,True
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,False,False,True
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10,True,False,False


Como é possível ver abaixo, há alguns problemas com os nomes dos atributos.

In [ ]:
df.columns

Index([' Length', ' Diameter', ' Height', ' Whole_weight', ' Shucked_weight',
       ' Viscera_weight', ' Shell_weight\t', ' Rings', 'Sex_F', 'Sex_I',
       'Sex_M'],
      dtype='object')

Vamos renomeá-los:

In [ ]:
df.rename(columns = {' Length': 'length', ' Diameter': 'diameter', ' Height': 'height', ' Whole_weight': 'whole_weight', ' Shucked_weight': 'shucked_weight',
       ' Viscera_weight': 'viscera_weight', ' Shell_weight\t': 'shell_weight', ' Rings': 'rings'}, inplace = True)

In [ ]:
df.columns

Index(['length', 'diameter', 'height', 'whole_weight', 'shucked_weight',
       'viscera_weight', 'shell_weight', 'rings', 'Sex_F', 'Sex_I', 'Sex_M'],
      dtype='object')

Selecionando os atributos preditores e alvo:

In [ ]:
X = df.drop(['rings'], axis = 1)
y = df['rings']

Teremos que "consertar" o formato de y:

In [ ]:
print(X.shape, y.shape)

(4177, 10) (4177,)


In [ ]:
y = y.values.reshape(-1,1)
print(y.shape)

(4177, 1)


# Subset Selection

Para fazermos subset selection, teremos que utilizar [loops](https://www.geeksforgeeks.org/loops-in-python/), estruturas de repetição.

Além disso, faremos o uso da função `combinations`, que será responsável por fazer as combinações necessárias para o subset selection.

Documentação: [itertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations)

In [ ]:
dicionario_r2 = {}

## Best Subset Selection

In [ ]:
len(X.columns)

10

Para usarmos a padronização dos dados em uma validação cruzada, teremos que fazer uso da função `Pipeline()`

Documentação: [range()](https://www.w3schools.com/python/ref_func_range.asp)

Documentação: [pipeline()](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
linear = LinearRegression()
scaler = StandardScaler()

# Faça a padronização dos dados e depois a regressão linear
pipe = Pipeline([('scaler', scaler), ('estimator', linear)] )

# Melhor r2_score para aquele número de atributos
r2_cols = []
cols = []

# Melhor r2_score em geral
r2_best = 0
# Atributos desse melhor modelo
cols_best = []

# Loop que vai de 1 até 11
for i in range(1, len(X.columns) + 1):

  r2_ = 0
  cols_ = []

  for comb in combinations(X.columns, i):

    scores = cross_val_score(pipe, X[list(comb)], y, cv = 5, scoring = 'r2')

    if scores.mean() > r2_:
      #Melhor r2_score em geral
      r2_ = scores.mean()
      cols_ = comb

  # No final desse loop, quero que o melhor r2_score daquele número de atributos seja adicionado a r2_cols
  r2_cols.append([r2_, cols_])

  if r2_ > r2_best:
    #Melhor r2_score em geral e sua combinação de atributos
    r2_best = r2_
    cols_best = cols_


print(cols_best)
print(r2_best)

('diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'Sex_I')
0.41937919963555076


Em `r2_cols`, podemos ver a melhor combinação de atributos com o respectivo número de variáveis.

In [ ]:
r2_cols

[[0.24303049045150216, ('shell_weight',)],
 [0.3683737099818569, ('shucked_weight', 'shell_weight')],
 [0.3998631605820431, ('shucked_weight', 'shell_weight', 'Sex_I')],
 [0.40681269675883486,
  ('diameter', 'shucked_weight', 'shell_weight', 'Sex_I')],
 [0.410548133863743,
  ('diameter', 'whole_weight', 'shucked_weight', 'shell_weight', 'Sex_I')],
 [0.4174532775495775,
  ('diameter',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_I')],
 [0.41937919963555076,
  ('diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_I')],
 [0.41924273048650873,
  ('diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_F',
   'Sex_M')],
 [0.4186023342895323,
  ('length',
   'diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_F',
   'Sex_I')],
 [0.41860233428953225,
  ('length',
   'diameter

Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['Best Subset Selection'] = r2_best

## Forward Stepwise

In [ ]:
linear = LinearRegression()
scaler = StandardScaler()

"""
Aqui está a diferença desse código para o código de best subset selection:
Toda vez que o loop interno acabar, irei forçar que o atributo que fez a função performar melhor
esteja na próxima seleção de melhor atributo
"""
atributos = []
atributos_restantes = list(X.columns.values)


# Faça a padronização dos dados e depois a regressão linear
pipe = Pipeline([('scaler', scaler), ('estimator', linear)] )

# Melhor r2_score e o número de atributos
r2_cols_forward = []

# Melhor r2_score em geral
r2_forward = 0
# Atributos desse melhor modelo
cols_forward = []

# Loop que vai de 1 até 11
for i in range(1, len(X.columns.values) + 1):

  r2_ = 0
  cols_ = []

  for comb in combinations(atributos_restantes, 1):

    scores = cross_val_score(pipe, X[list(comb) + atributos], y, cv = 5, scoring = 'r2')

    if scores.mean() > r2_:
      #Melhor r2_score em geral
      r2_ = scores.mean()
      cols_ = list(comb) + atributos
      atributo_ = comb[0]

  # No final desse loop interno, quero que o melhor r2_score daquele número de atributos seja adicionado a r2_cols_forward
  r2_cols_forward.append([r2_, cols_])
  # E que esses atributos sejam retirados dos atributos_restantes
  atributos_restantes.remove(atributo_)
  atributos.append(atributo_)


  if r2_ > r2_forward:
    #Melhor r2_score em geral e sua combinação de atributos
    r2_forward = r2_
    cols_forward = cols_


print(cols_forward)
print(r2_forward)

['height', 'shell_weight', 'shucked_weight', 'Sex_I', 'diameter', 'whole_weight', 'viscera_weight']
0.41937919963555076


Em `r2_cols_forward`, podemos ver a melhor combinação de atributos com o respectivo número de variáveis.

In [ ]:
r2_cols_forward

[[0.24303049045150216, ['shell_weight']],
 [0.3683737099818569, ['shucked_weight', 'shell_weight']],
 [0.39986316058204296, ['Sex_I', 'shell_weight', 'shucked_weight']],
 [0.4068126967588349, ['diameter', 'shell_weight', 'shucked_weight', 'Sex_I']],
 [0.41054813386374295,
  ['whole_weight', 'shell_weight', 'shucked_weight', 'Sex_I', 'diameter']],
 [0.41745327754957773,
  ['viscera_weight',
   'shell_weight',
   'shucked_weight',
   'Sex_I',
   'diameter',
   'whole_weight']],
 [0.41937919963555076,
  ['height',
   'shell_weight',
   'shucked_weight',
   'Sex_I',
   'diameter',
   'whole_weight',
   'viscera_weight']],
 [0.4192427304865086,
  ['Sex_F',
   'shell_weight',
   'shucked_weight',
   'Sex_I',
   'diameter',
   'whole_weight',
   'viscera_weight',
   'height']],
 [0.41860233428953253,
  ['length',
   'shell_weight',
   'shucked_weight',
   'Sex_I',
   'diameter',
   'whole_weight',
   'viscera_weight',
   'height',
   'Sex_F']],
 [0.41722301333387896,
  ['Sex_M',
   'shell_wei

Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['Forward Stepwise Selection'] = r2_forward

## Backward Stepwise

In [ ]:
linear = LinearRegression()
scaler = StandardScaler()

"""
Aqui está a diferença desse código para o código de best subset selection:
Toda vez que o loop interno acabar, irei fazer com que a variável menos
significativa seja excluída
"""

atributos_restantes = list(X.columns.values)


# Faça a padronização dos dados e depois a regressão linear
pipe = Pipeline([('scaler', scaler), ('estimator', linear)] )

# Melhor r2_score e o número de atributos
r2_cols_backward = []

# Melhor r2_score em geral
r2_backward = 0

# Atributos desse melhor modelo
cols_backward = []

# Loop que vai de 10 até 1
for i in range(len(X.columns.values), 0, -1):

  r2_ = 0
  cols_ = []

  for comb in combinations(atributos_restantes, i):

    scores = cross_val_score(pipe, X[list(comb)], y, cv = 5, scoring = 'r2')

    if scores.mean() > r2_:
      #Melhor r2_score em geral
      r2_ = scores.mean()
      cols_ = list(comb)

  # No final desse loop interno, quero que o melhor r2_score daquele número de atributos seja adicionado a r2_cols_backward
  r2_cols_backward.append([r2_, cols_])

  # E que esses atributos substituam os atributos_restantes
  atributos_restantes = cols_

  if r2_ > r2_backward:
    #Melhor r2_score em geral e sua combinação de atributos
    r2_backward = r2_
    cols_backward = cols_


print(cols_backward)
print(r2_backward)

['diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'Sex_I']
0.41937919963555076


In [ ]:
r2_cols_backward

[[0.41860233428953225,
  ['length',
   'diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_F',
   'Sex_I',
   'Sex_M']],
 [0.4186023342895323,
  ['length',
   'diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_F',
   'Sex_I']],
 [0.41924273048650845,
  ['diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_F',
   'Sex_I']],
 [0.41937919963555076,
  ['diameter',
   'height',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_I']],
 [0.4174532775495775,
  ['diameter',
   'whole_weight',
   'shucked_weight',
   'viscera_weight',
   'shell_weight',
   'Sex_I']],
 [0.410548133863743,
  ['diameter', 'whole_weight', 'shucked_weight', 'shell_weight', 'Sex_I']],
 [0.40681269675883486,
  ['diameter', 'shucked_weight', 'shell_weight', 'Sex_I']],
 [0.3998631605820431, ['shucked_weig

Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['Backward Stepwise Selection'] = r2_backward

# Regularização

## Ridge Regression

O funcionamento do código de Ridge Regression é bem similar ao de regressão linear.

O diferencial é o parâmetro de afinação ou penalidade λ. Por [lambda](https://www.w3schools.com/python/python_lambda.asp) ser uma palavra reservada em Python, alpha é o parâmetro que corresponde a essa penalidade no código.

Para usarmos cross-validation, podemos utilizar o `cross_val_score` ou podemos utilizar o `RidgeCV`.

Qual a diferença de usar um ou outro? Com o RidgeCV não conseguiremos padronizar os dados sem vazar informação do treinamento para o teste.

Quando padronizamos os dados de treinamento e teste juntos, estamos dando informações da média e distribuição do data set de treinamento para as observações de teste. Isso é algo que geralmente queremos evitar para diminuir o overfitting dos dados.

Documentação: [Ridge Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

Documentação: [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV)


In [ ]:
from sklearn.linear_model import Ridge

Abaixo está a maneira mais simples de utilizar a função. Perceba, porém, que ela não utiliza de uma abordagem de validação cruzada, então seu resultado de R² pode ser um sobreajuste (overfitting).

In [ ]:
# Chamando minha função e determinando seus parâmetros
ridge = Ridge(alpha = 1.0)

# Ajustando o modelo
ridge.fit(X, y)

# Medindo sua acurácia
ridge.score(X, y)

0.535485070082645

### Cross-Validation

Note que o valor de R² quando fazemos uma validação cruzada é menor.

In [ ]:
# Chamando minha função e determinando seus parâmetros
ridge = Ridge(alpha = 1.0)

# Validação Cruzada no RidgeRegression
cross = cross_val_score(ridge, X, y, cv = 5)

# Média dos valores de R² da validação cruzada
cross.mean()

0.4270936572522762

### Pipeline - Padronização

Nos códigos anteriores, fizemos a regressão com os dados brutos, ou seja, com os dados não padronizados. Faremos agora uma pipeline de uma regressão com padronização.

In [ ]:
# Chamando minha função e determinando seus parâmetros
ridge = Ridge(alpha = 1.0)

# Padronização
scaler = StandardScaler()

# Comando que vai sequenciar os passos
pipe = Pipeline([('scaler', scaler), ('estimator', ridge)])

# Validação cruzada com Ridge Regression e os dados padronizados
cross = cross_val_score(pipe, X, y, cv = 5)

# Média dos valores de R² da validação cruzada
cross.mean()

0.41937750551538927

Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['Ridge Regression'] = cross.mean()

Veremos outro método mais para frente, mas uma das formas de se encontrar o parâmetro de afinação λ é com o uso de loops.

In [ ]:
# Lista com valores de lambda para o parâmetro de afinação
lambdas = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]

# Melhor lambda
best_lambda = 0

# Chamando minha função e determinando seus parâmetros


# Padronização
scaler = StandardScaler()

for i in lambdas:
  # Comando que vai sequenciar os passos
  pipe = Pipeline([('scaler', scaler), ('estimator', Ridge(alpha = i))])

  # Validação cruzada com Ridge Regression e os dados padronizados
  cross = cross_val_score(pipe, X, y, cv = 5)

  print(cross.mean(), i)

print(best_lambda)

0.41860319809750485 0.001
0.418610963813438 0.01
0.4186877799729814 0.1
0.41937750551538927 1
0.4218607360097124 10
0.4088624115197389 100
0.3037939174181604 1000
0


## Lasso

Novamente, o funcionamento do código é bem similar ao que vimos. Só devemos especificar o parâmetro de afinação.

Documentação: [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# Chamando minha função e determinando seus parâmetros
lass = Lasso(alpha = .001)

# Ajustando o modelo
lass.fit(X, y)

# Acurácia do modelo
lass.score(X, y)

0.5374194462275808

Note que provavelmente temos o mesmo problema do overfitting.

### Cross-Validation

Podemos usar `cross_val_score()` ou `cross_validate`.

O `LassoCV` é especialmente útil para descobrirmos o parâmetro de afinação, uma vez que ele nos permite achar o valor ótimo de `alpha`.

Documentação: [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV)

In [ ]:
from sklearn.linear_model import LassoCV

Este passo só é necessário porque a função pede que o formato de y seja (n, ), e não (n, 1)

In [ ]:
y = df['rings']
y.shape

(4177,)

In [ ]:
# Determinando os parâmetros e ajustando o modelo
lass = LassoCV(cv = 5, random_state = 23).fit(X, y)

# Performance do modelo
print(f"R²: {lass.score(X, y)}")

# Parâmetro de Afinação
print(f"λ: {lass.alpha_}")

R²: 0.5374356599569114
λ: 0.0009821270162846901


De fato, o valor de R² dos códigos supracitados estão com sobreajuste.

In [ ]:
# Lasso e seus parâmetros
lass = Lasso(alpha = 0.0009821270162846901)

# Cross-validation
cross = cross_val_score(lass, X, y, cv = 5)

# Performance do modelo
print(f"R²: {cross.mean()}")

R²: 0.42173396537382624


### Pipeline - Padronização

In [ ]:
# Lasso e seus parâmetros
lass = Lasso(alpha = 0.0009821270162846901)

# Padronização
scaler = StandardScaler()

# Pipeline
pipe = Pipeline([('scaler', scaler), ('estimator', lass)])

# Cross-validation com os dados padronizados
cross = cross_validate(pipe, X, y, cv = 5, return_estimator=True)

# Performance do modelo
print(f"R²: {cross['test_score'].mean()}")

R²: 0.41918241172547194


Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['Lasso'] = cross['test_score'].mean()

Com Cross-Validation, é mais difícil ter os coeficientes do nosso modelo, mas com o parâmetro `return_estimator = True` em `cross_validate` conseguimos acessar esses coeficientes:

In [ ]:
for model in cross['estimator']:
  print(model['estimator'].coef_)

[ 0.22979503  0.57141537  0.34973211  2.86489762 -3.30517497 -0.83619336
  1.58187039 -0.         -0.32656164  0.06395804]
[-0.05461624  1.26244815  0.46113092  4.69594271 -4.62856293 -1.1659268
  1.10257282 -0.         -0.36316016  0.0295254 ]
[-0.06684856  0.96801651  0.78303656  4.17948811 -4.19226623 -1.21844491
  1.06206773 -0.         -0.3967313   0.0078289 ]
[ 0.03797382  1.12671602  0.41965232  5.02634253 -4.82845629 -1.22866528
  1.0501538  -0.         -0.37967057  0.02867815]
[-1.54547182e-01  1.24721446e+00  3.89433560e-01  4.37748095e+00
 -4.39730461e+00 -1.15152055e+00  1.33071015e+00 -0.00000000e+00
 -4.00583404e-01  3.20126029e-03]


In [ ]:
X.columns

Index(['length', 'diameter', 'height', 'whole_weight', 'shucked_weight',
       'viscera_weight', 'shell_weight', 'Sex_F', 'Sex_I', 'Sex_M'],
      dtype='object')

# Redução de Dimensionalidade

Funcionamento praticamente igual aos outros:

1. Padronização (`StandardScaler()`);
2. Redução de Dimensionalidade (PCA ou PLS);
3. Regressão.

## PCA

Faremos o uso da `Pipeline` para sequenciar nossa ação.

Documentação: [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)



In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Determinando o número de componentes (parâmetros de PCA)
pca = PCA(n_components = 2)

# Regressão Linear
linear = LinearRegression()

# Padronização
scaler = StandardScaler()

# Pipeline
pipe = Pipeline(steps = [('scaler', scaler), ('reduce_dim', pca), ('linreg', linear)])

# Cross-Validation
cross = cross_validate(pipe, X, y, cv = 10)

In [ ]:
print(f"R²: {cross['test_score'].mean()}")

R²: 0.12210017339005749


O desempenho desse modelo foi péssimo!

Podemos ver se o número de componentes foi o responsável usando o `GridSearchCV`, uma função que permite encontrar o melhor parâmetro de afinação por meio da validação cruzada.

Documentação: [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Determinando os parâmetros que queremos encontrar o valor ótimo
parameters = {'pca__n_components': range(1, 9)}

# Determinando o número de componentes (parâmetros de PCA)
pca = PCA()

# Regressão Linear
linear = LinearRegression()

# Padronização
scaler = StandardScaler()

# Pipeline
pipe = Pipeline(steps = [('scaler', scaler), ('pca', pca), ('linreg', linear)])

# GridSearchCV
search = GridSearchCV(estimator = pipe, param_grid = parameters)

search.fit(X, y)

print(search.best_params_)

{'pca__n_components': 7}


In [ ]:
# Cross-validation com os parâmetros de GridSearchCV
cross = cross_validate(search, X, y, cv = 5)

In [ ]:
print(f"R²: {cross['test_score'].mean()}")

R²: 0.40654397796303865


Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['PCA'] = cross['test_score'].mean()

## PLS

Mesmo funcionamento do PCR, com a exceção de que não precisamos usar uma regressão linear para ajustarmos o modelo. Isto é, o método `PLSRegression` já faz a redução de dimensionalidade e treina o modelo.

Documentação: [PLS](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html#sklearn.cross_decomposition.PLSRegression)

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
# Determinando os parâmetros que queremos encontrar o valor ótimo
parameters = {'pls__n_components': range(1, 9)}

# Determinando o número de componentes (parâmetros de PLS)
pls = PLSRegression()

# Padronização
scaler = StandardScaler()

# Pipeline
pipe = Pipeline(steps = [('scaler', scaler), ('pls', pls)])

# GridSearchCV
search = GridSearchCV(estimator = pipe, param_grid = parameters)

search.fit(X, y)

print(search.best_params_)

{'pls__n_components': 6}


In [ ]:
cross = cross_validate(search, X, y, cv = 5)

In [ ]:
print(f"R²: {cross['test_score'].mean()}")

R²: 0.41269448492401467


Adicionando em um dicionário para comparação posterior.

In [ ]:
dicionario_r2['PLS'] = cross['test_score'].mean()

# Comparação entre Modelos

In [ ]:
d = pd.Series(dicionario_r2)
print(d)

Best Subset Selection          0.419379
Forward Stepwise Selection     0.419379
Backward Stepwise Selection    0.419379
Ridge Regression               0.419378
Lasso                          0.419182
PCA                            0.406544
PLS                            0.412694
dtype: float64


# Desafio
Aplique esses conceitos no data set de [Diabetes](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) do scikit-learn.

# Referências

SICOTTE, X. Choosing the optimal model: Subset selection. Data Blog, 2018. Disponível em [Choosing the optimal model: Subset selection](https://xavierbourretsicotte.github.io/subset_selection.html).

SCIKIT-LEARN. Pipelines and composite estimators. Scikit-Learn, [s.d]. Disponível em [Pipelines and composite estimators](https://scikit-learn.org/stable/modules/compose.html#pipeline)

VAROQUAUX, G; et al. Pipelining: chaining a PCA and a logistic regression. Scikit-Learn, [s.d]. Disponível em [Pipelining: chaining a PCA and a logistic regression](https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html)

RUTECKI, M. GridSearchCV + KFold CV: The Right Way. Kaggle, 2023. Disponível em [GridSearchCV + KFold CV: The Right Way](https://www.kaggle.com/code/marcinrutecki/gridsearchcv-kfold-cv-the-right-way)

SCIKIT-LEARN. Principal Component Regression vs Partial Least Squares Regression. Scikit-Learn, [s.d]. Disponível em [Principal Component Regression vs Partial Least Squares Regression](https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html)